### Credit Score challenge with missing values

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.cbook import flatten
import math
from sklearn.ensemble import RandomForestClassifier
from datetime import datetime

In [2]:
train_set = pd.read_csv(r'Data/train.csv')
test_set = pd.read_csv(r'Data/test.csv')

cols_Vincent = train_set.columns[:97]
cols_Duong = train_set.columns[97:]

C:\Users\boy19\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3051: DtypeWarning: Columns (35,43) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\boy19\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3051: DtypeWarning: Columns (34,42) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
# store all duplicated columns
duplicated_cols_dict= {}
duplicated_cols_list_full = []


# THIS FUNCTION is to drop all duplicates columns
for i, col in enumerate(train_set.columns):
    duplicated_cols_list= []
    # if the col is already one of the keys
    # then skip
    if col in duplicated_cols_list_full:
        continue
    for c in train_set.columns:
        if (train_set[col].equals(train_set[c])):
            duplicated_cols_list.append(c)
            duplicated_cols_list_full.append(c)
                   
    if (len(duplicated_cols_list) > 1):
        duplicated_cols_dict[col] = duplicated_cols_list   
    
drop_cols = set(list(flatten(list(duplicated_cols_dict.values())))) - set(duplicated_cols_dict.keys())
train_set = train_set.drop(drop_cols, axis= 1)
test_set  = test_set.drop(drop_cols, axis= 1)

In [4]:
train_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53030 entries, 0 to 53029
Columns: 149 entries, id to Field_82
dtypes: float64(87), int64(3), object(59)
memory usage: 60.3+ MB


In [5]:
train_set = train_set[train_set.columns[:74]]

In [6]:
unique_count = train_set.nunique()

In [7]:
#Only field 13 have 1 unique value
unique_count[unique_count == 1]

Field_13    1
dtype: int64

In [8]:
train_set['Field_1'].iloc[0]
datetime.strptime(train_set['Field_1'].iloc[0], '%Y-%m-%dT%H:%M:%S.%fZ')

datetime.datetime(2019, 7, 24, 3, 39, 2, 854000)

In [9]:
train_set = train_set[['Field_3', 'Field_4', 'Field_10', 'Field_12', 'Field_13', 'Field_14', 'Field_16',
                      'Field_17', 'Field_19', 'Field_20', 'Field_21', 'Field_20', '']]

KeyError: "['', 'Field_16', 'Field_17', 'Field_14'] not in index"

In [ ]:
train_set['Field_67'].nunique()

In [ ]:
temp = train_set[['Field_38', 'Field_39']]

In [ ]:
temp = temp[~temp.Field_38.isin(['DN', 'TN'])]
temp['Field_38'] = temp['Field_38'].astype(float)

In [ ]:
for i in range(len(temp)):
        if(not math.isnan(float(temp['Field_38'].iloc[i])) and not math.isnan(float(temp['Field_39'].iloc[i]))):
            if(math.isnan(temp['Field_38'].iloc[i]) and not temp['Field_39'].iloc[i] == 0):
                print("Not inverted at: " +str(i))
            elif(temp['Field_38'].iloc[i] == 0 and not temp['Field_39'].iloc[i] == 1):
                print("Not inverted at: " +str(i))

In [ ]:
float('0')

In [ ]:
train_set['Field_4'].nunique()

In [ ]:
train = train_set[:42424]
test = train_set[42425:]

In [ ]:
#random forest
y = train['label']
X = train.drop(columns = ['label', 'id'])
clf = RandomForestClassifier(max_depth=2, random_state=0)
clf.fit(X, y)





In [ ]:
train_set['label']